<a href="https://colab.research.google.com/github/luiz10luiz10/projetoAiBuiderss/blob/main/_2_RAG_Textos_e_Frases_AI_Builders.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.9/386.9 kB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.2/325.2 kB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.0 MB/s eta 0:00:00


In [ ]:
from google.colab import userdata
userdata.get('openai_api_key')

import openai
import pandas as pd
import numpy as np

from openai import OpenAI
client = OpenAI(api_key=userdata.get('openai_api_key'))


## Importaremos um arquivo com uma lista de palavras

In [ ]:
df = pd.read_csv('pizzas_ingredientes.csv')
print(df)

          Nome da Pizza                                       Ingredientes
0            Margherita                      Tomate, Mussarela, Manjericão
1             Pepperoni                               Mussarela, Pepperoni
2        Quatro Queijos         Mussarela, Parmesão, Gorgonzola, Provolone
3            Portuguesa  Mussarela, Presunto, Ovo, Cebola, Pimentão, Az...
4             Calabresa                       Mussarela, Calabresa, Cebola
5   Frango com Catupiry                        Frango, Catupiry, Mussarela
6            Napolitana                        Tomate, Mussarela, Azeitona
7              Havaiana                       Mussarela, Presunto, Abacaxi
8           Vegetariana      Mussarela, Tomate, Pimentão, Azeitona, Cebola
9                  Atum                            Atum, Cebola, Mussarela
10             Mexicana         Mussarela, Carne Moída, Jalapeño, Pimentão
11              Caprese     Mussarela, Tomate, Manjericão, Azeite de Oliva
12          Alho e Óleo  

## Preparando Funções
- Get Embedding para transformar texto em vetores
- Cosine Similarity para buscar por similaridade nos vetores

https://en.wikipedia.org/wiki/Cosine_similarity


In [ ]:
def get_embedding(text, model="text-embedding-3-small"):
   text = text.replace("\n", " ")
   print("Text: ", text)
   response = client.embeddings.create(
       input=text,
       model=model
    )
   return response.data[0].embedding

def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

In [ ]:
df["embedding"] = df["Ingredientes"].apply(lambda x: get_embedding(x))
df.to_csv("embedded_pizzas_ingredientes.csv")

Text:  Tomate, Mussarela, Manjericão
Text:  Mussarela, Pepperoni
Text:  Mussarela, Parmesão, Gorgonzola, Provolone
Text:  Mussarela, Presunto, Ovo, Cebola, Pimentão, Azeitona
Text:  Mussarela, Calabresa, Cebola
Text:  Frango, Catupiry, Mussarela
Text:  Tomate, Mussarela, Azeitona
Text:  Mussarela, Presunto, Abacaxi
Text:  Mussarela, Tomate, Pimentão, Azeitona, Cebola
Text:  Atum, Cebola, Mussarela
Text:  Mussarela, Carne Moída, Jalapeño, Pimentão
Text:  Mussarela, Tomate, Manjericão, Azeite de Oliva
Text:  Mussarela, Alho, Azeite de Oliva
Text:  Molho Pesto, Mussarela, Tomate
Text:  Camarão, Mussarela, Tomate, Cebola, Alho


In [ ]:
busca = input("Digite uma palavra: ")

Digite uma palavra: pizzas com mussarela


In [ ]:
busca_vetor = get_embedding(busca)

Text:  pizzas com mussarela


In [ ]:
df["similarity"] = df["embedding"].apply(lambda x: cosine_similarity(x, busca_vetor))
df.sort_values("similarity", ascending=False)

,Nome da Pizza,Ingredientes,embedding,similarity
13,Pesto,"Molho Pesto, Mussarela, Tomate","[0.0008008241420611739, -0.005471860524266958,...",0.640757
1,Pepperoni,"Mussarela, Pepperoni","[-0.009283523075282574, -0.0289885513484478, -...",0.617466
2,Quatro Queijos,"Mussarela, Parmesão, Gorgonzola, Provolone","[-0.010491698980331421, -0.009639249183237553,...",0.605024
14,Camarão,"Camarão, Mussarela, Tomate, Cebola, Alho","[0.03242151066660881, -0.007081347517669201, 0...",0.547741
0,Margherita,"Tomate, Mussarela, Manjericão","[0.00037920422619208694, 0.031122738495469093,...",0.538871
11,Caprese,"Mussarela, Tomate, Manjericão, Azeite de Oliva","[-0.004945764783769846, 0.025522587820887566, ...",0.525053
12,Alho e Óleo,"Mussarela, Alho, Azeite de Oliva","[-0.0019337033154442906, 0.014779691584408283,...",0.519474
10,Mexicana,"Mussarela, Carne Moída, Jalapeño, Pimentão","[0.01758684776723385, 0.02502024918794632, -0....",0.512044
3,Portuguesa,"Mussarela, Presunto, Ovo, Cebola, Pimentão, Az...","[0.010393406264483929, -0.016795657575130463, ...",0.506171
4,Calabresa,"Mussarela, Calabresa, Cebola","[0.022102879360318184, -0.021931705996394157, ...",0.502350
